In [7]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

class ClfSwitcher(BaseEstimator):

    def __init__(self, estimator = LogisticRegression(),):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [2]:
clf=ClfSwitcher()

In [8]:
parameter_space = {
    'estimator': [LogisticRegression(), LinearSVC(),
                  KNeighborsClassifier(),GaussianNB(),RandomForestClassifier()]
}





In [9]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

In [12]:
search = GridSearchCV(clf , parameter_space, n_jobs=-1, cv=5)
search.fit(X, y)

print('Best model:\n', search.best_params_)

Best model:
 {'estimator': GaussianNB()}


In [13]:
search.cv_results_

{'mean_fit_time': array([0.04758496, 0.00639801, 0.00080018, 0.0013998 , 0.13055787]),
 'std_fit_time': array([0.00508256, 0.00101925, 0.00040009, 0.00048965, 0.00990775]),
 'mean_score_time': array([0.00059996, 0.00479856, 0.00299854, 0.00059986, 0.00819774]),
 'std_score_time': array([4.89862882e-04, 6.73298286e-03, 3.81469727e-07, 7.99894354e-04,
        9.79822455e-04]),
 'param_estimator': masked_array(data=[LogisticRegression(), LinearSVC(),
                    KNeighborsClassifier(), GaussianNB(),
                    RandomForestClassifier()],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'estimator': LogisticRegression()},
  {'estimator': LinearSVC()},
  {'estimator': KNeighborsClassifier()},
  {'estimator': GaussianNB()},
  {'estimator': RandomForestClassifier()}],
 'split0_test_score': array([1., 1., 1., 1., 1.]),
 'split1_test_score': array([1.        , 1.        , 1.        , 0.96666667, 1.        ])